In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import torch.optim as optim
import numpy as np
from torchvision.models import resnet18, ResNet18_Weights
from torchvision import transforms
from PIL import Image
import os
import matplotlib.pyplot as plt
from torch.nn.utils.rnn import pad_sequence
from torchmetrics import MeanAbsoluteError
from sklearn.preprocessing import StandardScaler
from torchvision.transforms import Resize, ToTensor
from torchvision.transforms.functional import to_pil_image
# from models.resnet import ResNet18VisualEncoder


In [ ]:

# Load images function
def load_images(folder):
    images = []
    transform = transforms.Compose([
        transforms.Resize((100, 100)),
        transforms.ToTensor(),
    ])
    for filename in os.listdir(folder):
        if filename.endswith('.png'):
            img = Image.open(os.path.join(folder, filename)).convert('RGB')
            img = transform(img)
            images.append(img)
    return torch.stack(images)


In [ ]:

# Load data
images = load_images('./maze_maps2')

# Load trajectories data
data = np.load('trajectories_1.npy', allow_pickle=True)  # Load trajectories data


In [ ]:

class ResNet18VisualEncoder(nn.Module):
    def __init__(self):
        super(ResNet18VisualEncoder, self).__init__()
        # self.resnet18 = resnet18(weights=ResNet18_Weights.DEFAULT)
        self.resnet18 = resnet18(weights=None)  # Initialize without pre-trained weights
        self.resnet18 = nn.Sequential(*list(self.resnet18.children())[:-2])  # Remove the last two layers
        # self.softmax_pooling = nn.AdaptiveAvgPool2d((1, 1))  # Add a softmax pooling layer
        self.softmax_pooling = nn.Softmax(dim=1)  # Add softmax layer with dim=1

    def forward(self, images):
        features = self.resnet18(images)
        pooled_features = self.softmax_pooling(features)
        pooled_features = torch.flatten(pooled_features, 1)  # Flatten the features
        return pooled_features


In [ ]:

def Resnet_Visual_Encoder(images):
    encoder = ResNet18VisualEncoder()
    latent_embeddings = encoder(images)
    return latent_embeddings
latent_embeddings = ResNet18VisualEncoder()(images)


In [ ]:

# Define the Pathl function
def Pathl(data):
    trajectories = [d['trajectory'] for d in data]
    lengths = [len(traj) for traj in trajectories]
    return lengths


In [ ]:

# Calculate lengths
lengths = Pathl(data)
path_lengths = lengths    


In [ ]:
# Define the add_gaussian_noise function
def add_gaussian_noise(image, noise_level, std):
    noise = torch.randn(image.shape) * std
    noisy_image = image + noise
    noisy_image = nn.functional.normalize(noisy_image, dim=0, p=2.0)  # Normalize across channel dimension
    return noisy_image, noise_level


In [ ]:


# Define the create_cosine_noise_schedule function
def create_cosine_noise_schedule(num_timesteps, std_min, std_max):
    steps = list(range(num_timesteps))
    noise_levels = [std_min + 0.5 * (std_max - std_min) * (1 + math.cos(math.pi * t / num_timesteps)) for t in steps]
    return noise_levels


In [ ]:

# Define the generate_noisy_trajectories function
def generate_noisy_trajectories(images, data, std_min, std_max, num_timesteps):
    latent_dim = latent_embeddings.shape[1]
    noisy_trajectories = []

    # Calculate the noise schedule
    noise_schedule = create_cosine_noise_schedule(num_timesteps, std_min, std_max)

    # Generate a noisy trajectory for each image
    for i in range(len(images)):
        image = images[i]
        trajectory = data[i]['trajectory']
        length = len(trajectory)

        # Generate noisy trajectory points
        noisy_trajectory = []
        for j in range(length):
            t = j / length
            t = int(t)
            std = noise_schedule[t]
            noisy_image, noise_level = add_gaussian_noise(image, t, std)
            noisy_trajectory.append(noisy_image)

        # Stack the noisy trajectory points
        noisy_trajectory = torch.stack(noisy_trajectory)

        # Add the noisy trajectory to the list
        noisy_trajectories.append(noisy_trajectory)

    return noisy_trajectories


In [ ]:
# Generate noisy trajectories
noisy_trajectories = generate_noisy_trajectories(images, data, std_min=0.1, std_max=0.5, num_timesteps=100)


In [17]:


# Create the noisy_maze_1 folder if it doesn't exist
if not os.path.exists('noisy_maze_2'):
    os.makedirs('noisy_maze_2')

# Save the noisy images to the noisy_maze_1 folder
for i, noisy_trajectory in enumerate(noisy_trajectories):
    for j, noisy_image in enumerate(noisy_trajectory):
        noisy_image = to_pil_image(noisy_image)
        noisy_image.save(f'noisy_maze_2/{i}_{j}.png')
